In [2]:
pip install backtrader

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.5/419.5 kB 22.0 MB/s eta 0:00:00


In [5]:
pip install quantstats

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00


In [8]:
from datetime import datetime
import backtrader as bt
import math
import quantstats as qs
import pandas_datareader as pdr
import time
import yfinance as yf

# SMA 전략
class SmaCross(bt.Strategy):
    params = dict(
        pfast=30,   # 단기 SMA
        pslow=200   # 장기 SMA
    )

    def __init__(self):
        self.ind = dict()
        for i, v in enumerate(self.datas):
            self.ind[v] = dict()
            self.ind[v]['sma1'] = bt.ind.SMA(v.close, period=self.p.pfast)  # 단기 SMA
            self.ind[v]['sma2'] = bt.ind.SMA(v.close, period=self.p.pslow)  # 장기 SMA

    def next(self):
        for i, v in enumerate(self.datas):
            if self.getposition(v).size == 0:  # 포지션이 없을 경우
                # 단기 SMA가 장기 SMA를 위로 돌파했을 경우
                if self.ind[v]['sma1'][0] >= self.ind[v]['sma2'][0] and self.ind[v]['sma1'][-1] < self.ind[v]['sma2'][-1]:
                    # 진입 가능 포지션 계산 (현금 / 현재 가격)
                    if self.broker.get_cash() == self.broker.get_value():
                        # 진입한 종목이 없을 시 현재 보유 현금중 절반을 매수함 (수수료 때문에 0.99를 곱해줌)
                        order_size = math.floor(self.broker.get_cash() / v.close[0] / 2 * 0.99)
                    else:
                        # 진입한 종목이 있을 경우 나머지 현금으로 모두 매수함 (수수료 때문에 0.99를 곱해줌)
                        order_size = math.floor(self.broker.get_cash() / v.close[0] * 0.99)
                    self.buy(data=v, size=order_size)  # 롱 포지션 진입
            # 장기 SMA가 단기 SMA를 아래로 돌파했을 경우
            elif self.ind[v]['sma1'][0] <= self.ind[v]['sma2'][0] and self.ind[v]['sma1'][-1] > self.ind[v]['sma2'][-1]:
                self.close(data=v)  # 포지션 종료

    def log(self, message):
        print(message)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            cur_date = order.data.datetime.date(0)
            if order.isbuy():
                self.log(f'{cur_date} [매수 주문 실행] 종목: {order.data._name} \t 수량: {order.size} \t 가격: {order.executed.price:.2f}')
            elif order.issell():
                self.log(f'{cur_date} [매도 주문 실행] 종목: {order.data._name} \t 수량: {order.size} \t 가격: {order.executed.price:.2f}')
            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log(f'{cur_date} 주문이 거부되었습니다. 종목: {order.data._name} \t 수량: {order.size} \t 가격: {order.executed.price:.2f}')


if __name__ == '__main__':
    cerebro = bt.Cerebro()
    cerebro.broker.setcommission(commission=0.003)  # 0.3% 수수료 설정
    cerebro.broker.setcash(10_000_000)

    print('시작 포트폴리오 가격: %.2f' % cerebro.broker.getvalue())

    # 데이터 피드 생성
    stock_list = ['005930.KS', '017670.KS']
    for stock in stock_list:
        data = bt.feeds.PandasData(dataname= yf.download('MSFT', '2011-01-01', '2020-12-31'))

        cerebro.adddata(data, name=stock)  # 데이터 피드 추가


    cerebro.addstrategy(SmaCross)  # 전략 추가
    cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')  # 결과 분석 추가

    results = cerebro.run()
    print('끝 포트폴리오 가격: %.2f' % cerebro.broker.getvalue())

    strat = results[0]
    pyfoliozer = strat.analyzers.getbyname('pyfolio')

    returns, positions, transactions, gross_lev = pyfoliozer.get_pf_items()
    returns.index = returns.index.tz_convert(None)

    # 간단한 결과 출력
    print(f'\n')
    print("Result:")
    cagr = qs.stats.cagr(returns)
    mdd = qs.stats.max_drawdown(returns)
    sharpe = qs.stats.sharpe(returns)
    print(f"SHARPE: {sharpe:.2f}")
    print(f"CAGR: {cagr * 100:.2f} %")
    print(f"MDD : {mdd * 100:.2f} %")

    cerebro.plot()

    # 자세한 결과 html 파일로 저장
    df = yf.download('^KS11', start= '2011-01-01',end='2020, 12, 31') # 벤치마크를 코스피 지수로 함
    qs.reports.html(returns, benchmark=df['Close'], output=f'SMA_MULTI.html', title='result')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

시작 포트폴리오 가격: 10000000.00


2012-01-09 [매수 주문 실행] 종목: 005930.KS 	 수량: 176093 	 가격: 28.05
2012-01-09 [매수 주문 실행] 종목: 017670.KS 	 수량: 176093 	 가격: 28.05
2012-10-16 [매도 주문 실행] 종목: 005930.KS 	 수량: -176093 	 가격: 29.45
2012-10-16 [매도 주문 실행] 종목: 017670.KS 	 수량: -176093 	 가격: 29.45
2013-04-24 [매수 주문 실행] 종목: 005930.KS 	 수량: 168757 	 가격: 30.62
2013-04-24 [매수 주문 실행] 종목: 017670.KS 	 수량: 168757 	 가격: 30.62
2015-02-17 [매도 주문 실행] 종목: 005930.KS 	 수량: -168757 	 가격: 43.97
2015-02-17 [매도 주문 실행] 종목: 017670.KS 	 수량: -168757 	 가격: 43.97
2015-05-19 [매수 주문 실행] 종목: 005930.KS 	 수량: 153238 	 가격: 47.56
2015-05-19 [매수 주문 실행] 종목: 017670.KS 	 수량: 153238 	 가격: 47.56
2015-09-10 [매도 주문 실행] 종목: 005930.KS 	 수량: -153238 	 가격: 43.12
2015-09-10 [매도 주문 실행] 종목: 017670.KS 	 수량: -153238 	 가격: 43.12
2015-10-19 [매수 주문 실행] 종목: 005930.KS 	 수량: 139805 	 가격: 47.42
2015-10-19 [매수 주문 실행] 종목: 017670.KS 	 수량: 139805 	 가격: 47.42
2016-06-06 [매도 주문 실행] 종목: 005930.KS 	 수량: -139805 	 가격: 51.99
2016-06-06 [매도 주문 실행] 종목: 017670.KS 	 수량: -139805 	 가격: 51.99
2016-08-01 [매수 주

[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['^KS11']: ValueError("time data '2020, 12, 31' does not match format '%Y-%m-%d'")
/usr/local/lib/python3.10/dist-packages/quantstats/utils.py:269: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  .pct_change()
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprec

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
from google.colab import files
files.download('SMA_MULTI.html')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>